In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline



In [9]:
df=pd.read_csv("insurance_data.csv")
df.head()


,age,affordibility,bought_insurance
0,22,1,0
1,25,0,0
2,47,1,1
3,52,0,0
4,46,1,1


In [10]:
y=df['bought_insurance']
x=df.drop('bought_insurance',axis='columns')
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
x_scaled=scaler.fit_transform(x)
x_scaled[:10]

array([[0.09090909, 1.        ],
       [0.15909091, 0.        ],
       [0.65909091, 1.        ],
       [0.77272727, 0.        ],
       [0.63636364, 1.        ],
       [0.86363636, 1.        ],
       [0.84090909, 0.        ],
       [0.95454545, 0.        ],
       [1.        , 1.        ],
       [0.97727273, 1.        ]])

In [14]:
from sklearn.model_selection import train_test_split    
x_train,x_test,y_train,y_test=train_test_split(x_scaled,y,test_size=0.2)
x_train.shape

(22, 2)

In [15]:
model=keras.Sequential([
    keras.layers.Dense(1,input_shape=(2,),activation='sigmoid',kernel_initializer='ones',bias_initializer='zeros')
])
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
model.fit(x_train,y_train,epochs=1000)

Epoch 1/1000


e:\Projects\VSC\.venv\lib\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 453ms/step - accuracy: 0.4091 - loss: 0.7706
Epoch 2/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.4091 - loss: 0.7701
Epoch 3/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.4091 - loss: 0.7695
Epoch 4/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.4091 - loss: 0.7690
Epoch 5/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.4091 - loss: 0.7684
Epoch 6/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.4091 - loss: 0.7679
Epoch 7/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.4091 - loss: 0.7673
Epoch 8/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.4091 - loss: 0.7668
Epoch 9/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.4091 - loss: 0.7662
Epoch 10/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - accuracy: 0.4091 - loss: 0.7657
Epoch 11/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.4091 - loss: 0.7651
Epoch 12/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 0.4091 - lo

In [16]:
y_pred=model.predict(x_test)
print(y_pred)
print(y_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
[[0.745703  ]
 [0.65832466]
 [0.62712526]
 [0.6261748 ]
 [0.5843962 ]
 [0.6670557 ]]
9     1
14    1
6     0
23    1
22    1
7     1
Name: bought_insurance, dtype: int64


In [ ]:
def sigmoid(x):
    return (1/(1+np.exp(-x)))
def logloss(y_true,y_pred):
    epsilon=1e-15
    y_pred_new=[max(i,epsilon) for i in y_pred]
    y_pred_new=[min(i,1-epsilon) for i in y_pred_new]
    y_pred_new=np.array(y_pred_new)
    ll=-np.mean(y_true*np.log(y_pred_new)+(1-y_true)*np.log(1-y_pred_new))
    return ll



class NeuralNetwork:
    def __init__(self):
        self.weights = None
        self.bias=None
        self.epoch_list=[]
        
    def fit(self,x,y,epochs,losstr):
        self.weights,self.bias=self.gradient_descent(x,y,epochs,losstr)
        print(f"Final Weights are {self.weights} and bias is {self.bias}    ")

    def gradient_descent(self,x,y,epochs,losstr):
        n_samples,n_features=x.shape
        weights=np.ones(n_features)
        bias=0
        learning_rate=0.01
        

        for i in range(epochs):
            w_sum=np.dot(x,weights)+bias
            y_pred=sigmoid(w_sum)
            loss_at_current_epoch=logloss(y,y_pred)

            w_gradient=(1/n_samples)*np.dot(x.T,(y_pred-y))
            b_gradient=(1/n_samples)*np.sum(y_pred-y)

            weights=weights-learning_rate*w_gradient
            bias=bias-learning_rate*b_gradient

            if(i%50==0):
                self.epoch_list.append((i,weights,bias,loss_at_current_epoch))


            if(loss_at_current_epoch<=losstr):
                self.epoch_list.append((i,weights,bias,loss_at_current_epoch))
                break
            
        return weights,bias
    def predict(self,x):
        w_sum=np.dot(x,self.weights)+self.bias
        y_pred=sigmoid(w_sum)
        y_pred=[1 if i>0.5 else 0 for i in y_pred]
        return np.array(y_pred)

In [17]:
custom_model=NeuralNetwork()
custom_model.fit(x_train,y_train,1000,0.4)

Final Weights are [1.37664435 0.57391522] and bias is -1.175500694739923    


In [20]:
coef,intercept=model.get_weights()
print(f"Custom Model Weights are {custom_model.weights} and bias is {custom_model.bias}    ")
print(f"Keras Model Weights are {coef} and bias is {intercept}    ")

Custom Model Weights are [1.37664435 0.57391522] and bias is -1.175500694739923    
Keras Model Weights are [[1.5399531 ]
 [0.34592688]] and bias is [-0.77505666]    


In [23]:
model.predict(x_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step


array([[0.745703  ],
       [0.65832466],
       [0.62712526],
       [0.6261748 ],
       [0.5843962 ],
       [0.6670557 ]], dtype=float32)

In [24]:
custom_model.predict(x_test)

array([1, 1, 0, 1, 1, 1])